In [1]:
sc

In [2]:
! ls -alh /data/tmp/

total 45G
drwxr-xr-x 2 ubuntu ubuntu 4.0K Oct 19 13:32 .
drwxr-xr-x 6 root   root   4.0K Oct 19 12:20 ..
-rw-rw-r-- 1 ubuntu ubuntu  14K Oct 19 12:59 log
-rw-rw-r-- 1 ubuntu ubuntu 639M Oct 19 13:26 pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 757M Oct 19 13:13 pr_day_BCSD_historical_r1i1p1_bcc-csm1-1_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 745M Oct 19 13:19 pr_day_BCSD_historical_r1i1p1_BNU-ESM_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 773M Oct 19 13:04 pr_day_BCSD_historical_r1i1p1_CanESM2_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 568M Oct 19 13:14 pr_day_BCSD_historical_r1i1p1_CCSM4_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 573M Oct 19 13:06 pr_day_BCSD_historical_r1i1p1_CESM1-BGC_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 622M Oct 19 13:21 pr_day_BCSD_historical_r1i1p1_CNRM-CM5_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 664M Oct 19 13:24 pr_day_BCSD_historical_r1i1p1_CSIRO-Mk3-6-0_1997.nc
-rw-rw-r-- 1 ubuntu ubuntu 666M Oct 19 13:08 pr_day_BCSD_historical_r1i1p1_GFDL-CM3_1997.nc
-rw-rw-r-- 1 u

In [2]:
files = !! ls /data/tmp
files = [f for f in files if f.endswith('nc')]

In [3]:
from netCDF4 import Dataset, num2date

In [4]:
ds = Dataset("/data/tmp/" + files[0], 'r', format='NETCDF4')

In [4]:
import numpy as np

np.shape(ds['pr'])

(365, 720, 1440)

### Example Avro
http://avro.apache.org/docs/1.7.7/gettingstartedpython.html

In [8]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import json

In [9]:
python_schema = {
    "namespace": "example.avro",
    "type": "record",
    "name": "User",
    "fields": [
         {"name": "name", "type": "string"},
         {"name": "favorite_number",  "type": ["int", "null"]},
         {"name": "favorite_color", "type": ["string", "null"]}
     ]
}

In [ ]:
schema = avro.schema.parse(json.dumps(python_schema))

In [ ]:
writer = DataFileWriter(open("users.avro", "w"), DatumWriter(), schema)
writer.append({"name": "Alyssa", "favorite_number": 256})
writer.append({"name": "Ben", "favorite_number": 7, "favorite_color": "red"})
writer.close()

In [ ]:
reader = DataFileReader(open("users.avro", "r"), DatumReader())
for user in reader:
    print user
reader.close()

### NetCDF test

In [1]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import json

In [11]:
python_schema = {
    "namespace": "com.kitware.nex_gddp",
    "type": "record",
    "name": "GDDP Record",
    "fields": [
#        {"name": "key", "type": "string"},
        {"name": "variable", "type": "float"},
        {"name": "lat",  "type": "float"},
        {"name": "lon", "type": "float"},
        {"name": "time", "type": "long"},
     ]
}

schema = avro.schema.parse(json.dumps(python_schema))

writer = DataFileWriter(open("{}.avro".format(files[0].split(".")[0]), 
                             "w"), DatumWriter(), schema)

### Scratch

In [5]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.

    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])

    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [6]:
ds

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format UNDEFINED):
    parent_experiment: pre-industrial control
    parent_experiment_id: piControl
    parent_experiment_rip: r1i1p1
    Conventions: CF-1.4
    institution: NASA Earth Exchange, NASA Ames Research Center, Moffett Field, CA 94035
    institute_id: NASA-Ames
    realm: atmos
    modeling_realm: atmos
    version: 1.0
    downscalingModel: BCSD
    experiment_id: historical
    frequency: day
    realization: 1
    initialization_method: 1
    physics_version: 1
    tracking_id: f7735636-28ce-46f3-8474-09fd3dd4607b
    driving_data_tracking_ids: N/A
    driving_model_ensemble_member: r1i1p1
    driving_experiment_name: pre-industrial control
    driving_experiment: piControl
    model_id: BCSD
    references: BCSD method: Thrasher et al., 2012, Hydrol. Earth Syst. Sci.,16, 3309-3314. 
Ref period obs: latest version of the Princeton Global Meteorological Forcings (http://hydrology.princeton.ed

In [9]:
import time
import numpy as np

f = np.vectorize(lambda x: int(time.mktime(x.timetuple())))

time_dim = f(num2date(ds.variables['time'][:], ds.variables['time'].units))

dims = [time_dim, ds['lat'], ds['lon']]
        
zeros = np.zeros((np.prod([x.size for x in dims]), len(dims) + 1))


#### Memory error

In [10]:
cartesian(dims, out=zeros)

array([[(-2800, inf, inf, 851965184.0), (-89, -89.875, -89.875, -89.875),
        (0, 0.125, 0.125, 0.125), (0, 0.0, 0.0, 0.0)],
       [(-2800, inf, inf, 851965184.0), (-89, -89.875, -89.875, -89.875),
        (0, 0.375, 0.375, 0.375), (0, 0.0, 0.0, 0.0)],
       [(-2800, inf, inf, 851965184.0), (-89, -89.875, -89.875, -89.875),
        (0, 0.625, 0.625, 0.625), (0, 0.0, 0.0, 0.0)],
       ..., 
       [(-10480, inf, inf, 883414784.0), (89, 89.875, 89.875, 89.875),
        (359, 359.5, 359.5, 359.375), (0, 0.0, 0.0, 0.0)],
       [(-10480, inf, inf, 883414784.0), (89, 89.875, 89.875, 89.875),
        (359, 359.5, 359.5, 359.625), (0, 0.0, 0.0, 0.0)],
       [(-10480, inf, inf, 883414784.0), (89, 89.875, 89.875, 89.875),
        (359, 360.0, 360.0, 359.875), (0, 0.0, 0.0, 0.0)]], 
      dtype=[('time', '<i2'), ('lat', '<f2'), ('lon', '<f2'), ('pr', '<f4')])